In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
import json

In [ ]:
model_name = "Shodai1122/gemma-2-2b-it"

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", #input your hg key here
)
FastLanguageModel.for_inference(model)

In [ ]:
from datasets import load_dataset
datasets = load_dataset('neural-bridge/rag-dataset-12000', split='test')
print(f"Train dataset size: {len(datasets)} ")

In [ ]:
datasets = datasets.select(range(100))
print(len(datasets))

In [ ]:
print(datasets[0])

In [ ]:
from tqdm import tqdm

# 推論
results = []
for dt in tqdm(datasets):
  context=dt["context"]
  question = dt["question"]
  answer = dt["answer"]

  prompt = f"""Given the following passage, answer the related question.\n### Passage\n{context}\n### Question\n{question}\n### Answer\n"""

  inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, do_sample=False, repetition_penalty=1.2)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### Answer')[-1]

  results.append({"question": question, "output": prediction, "answer": answer})

In [ ]:
with open(f"gemma-2-2b-it_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')